<a href="https://colab.research.google.com/github/SoungbinC/CMPE294_Assignments/blob/Assignment_Transformer/nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Topic: Building nanogpt from scratch

In [15]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://gutenberg.net.au/ebooks02/0200041.txt -O gatsby.txt

--2024-12-03 10:14:56--  https://gutenberg.net.au/ebooks02/0200041.txt
Resolving gutenberg.net.au (gutenberg.net.au)... 203.210.102.143
Connecting to gutenberg.net.au (gutenberg.net.au)|203.210.102.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 269691 (263K) [text/plain]
Saving to: ‘gatsby.txt’

gatsby.txt          100%[===================>] 263.37K   315KB/s    in 0.8s    

2024-12-03 10:14:58 (315 KB/s) - ‘gatsby.txt’ saved [269691/269691]



## Download sample

In [18]:
# read it in to inspect it
with open('gatsby.txt', 'r', encoding='utf-8', errors='ignore') as f:
    text = f.read()


In [21]:
# Improved cleaning with proper space handling
# Remove non-relevant sections before "Chapter 1"
start_index = text.find('Chapter 1')
if start_index == -1:
    raise ValueError("Could not find the start of the book ('Chapter 1') in the text.")

cleaned_text = text[start_index:]  # Extract from "Chapter 1" onwards

# Additional cleaning
# Normalize spaces and ensure proper formatting
import re
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Replace multiple spaces/newlines with a single space
cleaned_text = re.sub(r'chapter\s+\d+', lambda x: x.group(0).capitalize() + "\n\n", cleaned_text)  # Format "Chapter X"

# Strip leading and trailing whitespace
cleaned_text = cleaned_text.strip()

# Save cleaned text to a file (optional)
with open('gatsby_cleaned.txt', 'w', encoding='utf-8') as f:
    f.write(cleaned_text)

# Preview cleaned text
print(cleaned_text[:500])  # Print the first 500 characters for inspection


Chapter 1 In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. "Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven't had the advantages that you've had." He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that ha


## Set up hypermeters

In [8]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

## Tokenize text

In [22]:
chars = sorted(list(set(cleaned_text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]  # String to integers
decode = lambda l: ''.join([itos[i] for i in l])  # Integers to string

# Convert the cleaned text to a tensor
data = torch.tensor(encode(cleaned_text), dtype=torch.long)

# Split into training and validation sets
n = int(0.9 * len(data))  # 90% training, 10% validation
train_data = data[:n]
val_data = data[n:]

## Train and test set

In [7]:
# Train and test splits
data = torch.tensor(encode(cleaned_text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## loading data

In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x



# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

In [24]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.211277 M parameters
step 0: train loss 4.6030, val loss 4.6064
step 100: train loss 2.6151, val loss 2.6157
step 200: train loss 2.4845, val loss 2.5025
step 300: train loss 2.4032, val loss 2.4245
step 400: train loss 2.3168, val loss 2.3130
step 500: train loss 2.2365, val loss 2.2545
step 600: train loss 2.1741, val loss 2.1964
step 700: train loss 2.1142, val loss 2.1301
step 800: train loss 2.0737, val loss 2.0910
step 900: train loss 2.0174, val loss 2.0362
step 1000: train loss 1.9736, val loss 2.0073
step 1100: train loss 1.9432, val loss 1.9751
step 1200: train loss 1.9173, val loss 1.9321
step 1300: train loss 1.8864, val loss 1.9094
step 1400: train loss 1.8779, val loss 1.9053
step 1500: train loss 1.8430, val loss 1.8800
step 1600: train loss 1.8243, val loss 1.8589
step 1700: train loss 1.8009, val loss 1.8341
step 1800: train loss 1.7812, val loss 1.8155
step 1900: train loss 1.7865, val loss 1.8147
step 2000: train loss 1.7622, val loss 1.8125
step 2100: train loss 1.